In [1]:
import numpy as np
import pyprind
import pandas as pd
import os
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.stem.porter import PorterStemmer

In [2]:
#开宇给大家的CODE开头我省略了，一些分词之类的，直接从他生成好的文件提取

count = CountVectorizer()
with open('movie_data.csv', encoding='utf-8', mode = 'r') as f:
    bag = count.fit_transform(f)
    print(count.vocabulary_)
    porter = PorterStemmer()
def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

{'review': 75508, 'sentiment': 80084, 'in': 44763, '1974': 536, 'the': 90160, 'teenager': 89502, 'martha': 56009, 'moxley': 60273, 'maggie': 54803, 'grace': 38432, 'moves': 60212, 'to': 91217, 'high': 41961, 'class': 17532, 'area': 5708, 'of': 63757, 'belle': 9319, 'haven': 40916, 'greenwich': 38816, 'connecticut': 19486, 'on': 64110, 'mischief': 58705, 'night': 62315, 'eve': 30788, 'halloween': 40127, 'she': 80841, 'was': 98149, 'murdered': 60663, 'backyard': 7574, 'her': 41628, 'house': 43198, 'and': 4541, 'murder': 60661, 'remained': 74499, 'unsolved': 95354, 'twenty': 93425, 'two': 93484, 'years': 100857, 'later': 51552, 'writer': 100377, 'mark': 55863, 'fuhrman': 35567, 'christopher': 16993, 'meloni': 57383, 'who': 99131, 'is': 46765, 'former': 34637, 'la': 50868, 'detective': 24505, 'that': 90137, 'has': 40745, 'fallen': 32009, 'disgrace': 25661, 'for': 34443, 'perjury': 67309, 'simpson': 82052, 'trial': 92544, 'moved': 60202, 'idaho': 44024, 'decides': 23141, 'investigate': 4649

In [3]:
nltk.download('stopwords')
from nltk.corpus import stopwords
#Classify the topics of movie set
df = pd.read_csv('movie_data.csv',encoding = 'utf-8')
count = CountVectorizer(stop_words = 'english',max_df=.15,max_features=5000)
X = count.fit_transform(df['review'].values)
from sklearn.decomposition import LatentDirichletAllocation
lda = LatentDirichletAllocation(n_components = 10, random_state = 0, learning_method='batch')
X_topics = lda.fit_transform(X)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\David\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
n_top_words = 30
feature_names = count.get_feature_names()
feature_vec = [None]
for topic_idx,topic in enumerate(lda.components_):
    print("Topic %d:" % (topic_idx+1))
    print(" ".join([feature_names[i] for i in topic.argsort()\
        [:-n_top_words-1:-1]]))   
    feature_vec.append ((" ".join([feature_names[i] for i in topic.argsort()\
        [:-n_top_words-1:-1]]))) 

feature_vec.pop(0)
feature_vec = np.array(feature_vec)
feature_vec = count.fit_transform(feature_vec)
print(count.vocabulary_)

from sklearn.feature_extraction.text import TfidfTransformer
tfidf = TfidfTransformer(use_idf = True,norm ='l2',smooth_idf = True)
np.set_printoptions(precision = 4)
feature_vec = tfidf.fit_transform(feature_vec)# coded feature vectors
print(feature_vec.toarray())

Topic 1:
comedy funny cast role actors script actor performance doesn play work original director humor quite jokes isn makes laugh hilarious plays comic moments roles lines screen bit played times hard
Topic 2:
world point real director war fact human audience american work interesting making men documentary things makes feel women doesn sense message far political main self society viewer kind reality subject
Topic 3:
action john music musical role star big song dance cast new james songs plays western played fun classic old hollywood years screen dancing stars stewart performance early singing real actor
Topic 4:
horror performance director beautiful work quite performances excellent cast dark makes music atmosphere wonderful 10 perfect young role truly brilliant cinematography especially thriller moments style score tale actors interesting bit
Topic 5:
woman gets house girl guy sex murder wife women police young killer old goes town doesn black pretty husband night dead killed play

In [5]:
print(X_topics)

[[0.0011 0.0011 0.0011 ... 0.0011 0.0011 0.0851]
 [0.0014 0.0014 0.0014 ... 0.9871 0.0014 0.0014]
 [0.3383 0.0368 0.0011 ... 0.2716 0.0011 0.1322]
 ...
 [0.0028 0.0028 0.0974 ... 0.416  0.0028 0.0028]
 [0.0036 0.0036 0.0036 ... 0.0036 0.8732 0.0036]
 [0.5889 0.0067 0.0067 ... 0.0067 0.0067 0.3578]]


In [22]:
feature_vec[0][0]

<1x135 sparse matrix of type '<class 'numpy.float64'>'
	with 10 stored elements in Compressed Sparse Row format>

In [103]:
feature_list=feature_vec.toarray()
df_reclist=pd.read_excel('Rec_list.xlsx')

COS_list=[]
from scipy.spatial import distance

#这里的input_data_trained是随便生成的，实际中应该是用户INPUT之后模型训练完的一个特征向量
#注意需要大小为 1 X 135
input_data_trained=np.zeros(135) 
input_data_trained[5:20]=0.995

i=0
b=input_data_trained.reshape(-1,1)

# 为了保证余弦对数据敏感，用a和b减去自身的平均数后再算余弦相似度（1-COSINE DISTANCE)
b=b-b.mean()
while i<10:
    a=feature_list[i].reshape(-1, 1)
    a=a-a.mean()
    COS_list.append(1-distance.cosine(a,b))
    i=i+1
    
index_num=COS_list.index(max(COS_list))
print("Recommendations: ")
print(df_reclist.iloc[:,index_num])





The World at War
Recommendations: 
0                           Parasite
1                       The Shining 
2                              Alien
3    What Ever Happened to Baby Jane
4                         The Thing 
Name: 4, dtype: object
